# Análise Spotify

In [1]:
# Importe das bibliotecas
import pandas as pd

In [3]:
# Leitura dos dados dos arquivos historico extendido
data_2021_2023_audio = pd.read_json("./data/extended/Streaming_History_Audio_2021-2023_0.json")
data_2023_2024_audio = pd.read_json("./data/extended/Streaming_History_Audio_2023-2024_1.json")
data_2024_2025_audio = pd.read_json("./data/extended/Streaming_History_Audio_2024-2025_2.json")
data_2022_2025_video = pd.read_json("./data/extended/Streaming_History_Video_2022-2025.json")

In [5]:
# Leitura dos dados dos arquivos da conta

data_stremingHistoricalMusic = pd.read_json("./data/account/StreamingHistory_music_0.json")
data_stremingHistoricalPodcast = pd.read_json("./data/account/StreamingHistory_podcast_0.json")

In [6]:
# Empinha os dados

data_audio = pd.concat([data_2021_2023_audio, data_2023_2024_audio, data_2024_2025_audio])
# data_video = data_2022_2025_video

display(data_audio.head(5))
# display(data_video.head(5))

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2021-12-01T18:42:02Z,Windows 10 (10.0.22000; x64; AppX),0,BR,186.220.101.158,iTrack - Radio Edit,Tim Berg,iTrack & LoopeDe,spotify:track:6YXzDf0eK8QTbrdM6hx56s,None,...,NaN,NaN,NaN,playbtn,unexpected-exit,False,False,False,NaN,False
1,2021-12-01T18:46:22Z,Windows 10 (10.0.22000; x64; AppX),6860,BR,186.220.101.158,iTrack - Radio Edit,Tim Berg,iTrack & LoopeDe,spotify:track:6YXzDf0eK8QTbrdM6hx56s,None,...,NaN,NaN,NaN,playbtn,endplay,False,False,False,NaN,False
2,2021-12-01T19:12:49Z,Windows 10 (10.0.22000; x64; AppX),0,BR,186.220.101.158,Levels - Radio Edit,Avicii,Levels,spotify:track:5UqCQaDshqbIk3pkhy4Pjg,None,...,NaN,NaN,NaN,clickrow,unexpected-exit,False,False,False,NaN,False
3,2021-12-01T19:12:49Z,Windows 10 (10.0.22000; x64; AppX),185730,BR,186.220.101.158,Summer,Calvin Harris,Motion,spotify:track:6YUTL4dYpB9xZO5qExPf05,None,...,NaN,NaN,NaN,clickrow,endplay,False,False,False,NaN,False
4,2021-12-01T19:16:24Z,Windows 10 (10.0.22000; x64; AppX),199906,BR,186.220.101.158,Levels - Radio Edit,Avicii,Levels,spotify:track:5UqCQaDshqbIk3pkhy4Pjg,None,...,NaN,NaN,NaN,clickrow,trackdone,False,False,False,NaN,False


In [7]:
# Remove colluns desnecessary

data_audio_clean = data_audio.drop(columns=["conn_country","ip_addr","spotify_episode_uri","audiobook_title","audiobook_uri","audiobook_chapter_uri","audiobook_chapter_title"])
# data_video_clean = data_video.drop(columns=["conn_country","ip_addr",])

In [8]:
# Create 3 columns year, mounth and day from ts column

#Convert ts column to datetime
data_audio_clean['ts'] = pd.to_datetime(data_audio_clean['ts']).dt.tz_localize(None)

#Create columns year
data_audio_clean['year'] = data_audio_clean['ts'].dt.year

#Create columns mounth with zero left

data_audio_clean['mounth'] = data_audio_clean['ts'].dt.strftime('%m')

#Create columns day
data_audio_clean['day'] = data_audio_clean['ts'].dt.strftime('%d')

display(data_audio_clean)

,ts,platform,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,year,mounth,day
0,2021-12-01 18:42:02,Windows 10 (10.0.22000; x64; AppX),0,iTrack - Radio Edit,Tim Berg,iTrack & LoopeDe,spotify:track:6YXzDf0eK8QTbrdM6hx56s,None,None,playbtn,unexpected-exit,False,False,False,NaN,False,2021,12,01
1,2021-12-01 18:46:22,Windows 10 (10.0.22000; x64; AppX),6860,iTrack - Radio Edit,Tim Berg,iTrack & LoopeDe,spotify:track:6YXzDf0eK8QTbrdM6hx56s,None,None,playbtn,endplay,False,False,False,NaN,False,2021,12,01
2,2021-12-01 19:12:49,Windows 10 (10.0.22000; x64; AppX),0,Levels - Radio Edit,Avicii,Levels,spotify:track:5UqCQaDshqbIk3pkhy4Pjg,None,None,clickrow,unexpected-exit,False,False,False,NaN,False,2021,12,01
3,2021-12-01 19:12:49,Windows 10 (10.0.22000; x64; AppX),185730,Summer,Calvin Harris,Motion,spotify:track:6YUTL4dYpB9xZO5qExPf05,None,None,clickrow,endplay,False,False,False,NaN,False,2021,12,01
4,2021-12-01 19:16:24,Windows 10 (10.0.22000; x64; AppX),199906,Levels - Radio Edit,Avicii,Levels,spotify:track:5UqCQaDshqbIk3pkhy4Pjg,None,None,clickrow,trackdone,False,False,False,NaN,False,2021,12,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4779,2025-06-21 15:33:39,android,2701,I Lived,OneRepublic,Native,spotify:track:3IQF4xCQUPicbA4hWfTxPo,None,None,fwdbtn,fwdbtn,True,True,False,1.750520e+09,False,2025,06,21
4780,2025-06-21 15:33:44,android,4865,The Man Who Can't Be Moved,The Script,The Script,spotify:track:3nqqDo8CcCLke3ZoTgiOKf,None,None,fwdbtn,fwdbtn,True,True,False,1.750520e+09,False,2025,06,21
4781,2025-06-21 15:43:16,android,4527,Cold Water (feat. Justin Bieber & MØ),Major Lazer,Cold Water (feat. Justin Bieber & MØ),spotify:track:6Bjtr3tPdto5nyuLYf61sN,None,None,fwdbtn,logout,True,False,False,1.750520e+09,False,2025,06,21
4782,2025-06-21 16:31:40,android,182379,Cold Water (feat. Justin Bieber & MØ),Major Lazer,Cold Water (feat. Justin Bieber & MØ),spotify:track:6Bjtr3tPdto5nyuLYf61sN,None,None,appload,trackdone,True,False,False,1.750523e+09,False,2025,06,21


In [9]:
# Freq. Plataform 

display(data_audio_clean['platform'].value_counts())

platform
android                                                                             23628
Android OS 11 API 30 (motorola, moto g(9) plus)                                      9020
windows                                                                              1122
not_applicable                                                                       1007
Windows 10 (10.0.22000; x64; AppX)                                                    839
ios                                                                                   411
Windows 10 (10.0.22622; x64; AppX)                                                     94
Partner amazon_salmon Amazon;Echo_Show_5;27d4dfe427b34d57995b463e5d63198d;;tpapi       61
web_player android 11;chrome 98.0.4758.87;mobile                                        1
Name: count, dtype: int64

In [10]:
# cria coluna unica de plataforma

data_audio_clean['platform_clean'] = data_audio_clean['platform'].apply(
    lambda x: (
        "Android" if "android" in x.lower() else
        "Windows" if "windows" in x.lower() else
        "IOS" if "ios" in x.lower() else
        "desconhecido" if "not_applicable" in x.lower() else
        None
    )
)

In [11]:
# Export the cleaned data to Excel files
data_audio_clean.to_excel("./streaming_history_audio_clean.xlsx", index=False)
data_stremingHistoricalMusic.to_excel("./streaming_history_music_clean.xlsx", index=False)
data_stremingHistoricalPodcast.to_excel("./streaming_history_podcast_clean.xlsx", index=False)